##### 固有空間の次元数などの定数値の設定

In [12]:
import os


# 訓練データセット（画像ファイルリスト）のファイル名
DATASET_CSV = './tinyCelebA/train_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinyCelebA'

# 画像サイズ
H = 128 # 縦幅
W = 128 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 読み込む画像の枚数
N_SAMPLES = 2048

# 固有空間の次元数
N_DIMS = 64

# GMMにおけるコンポーネント分布数
N_COMPONENTS = 50

# PCA結果などの保存先フォルダ
MODEL_DIR = './PCA_based'

# PCA結果の保存先
RESULT_FILE_MEAN = os.path.join(MODEL_DIR, 'mean.npy') # 平均ベクトル
RESULT_FILE_EIGENVECTORS = os.path.join(MODEL_DIR, 'eigenvectors.npy') # 固有ベクトル
RESULT_FILE_COMPRESSED_DATA = os.path.join(MODEL_DIR, 'compressed_data.npy') # 圧縮後データ

# GMM当てはめ結果の保存先
RESULT_FILE_GMM = os.path.join(MODEL_DIR, 'gmm.pkl')

##### 訓練データセットの読み込み
- 数十秒かかる可能性があります

In [2]:
import numpy as np
from PCA_based.func import make_list, read_images


# 画像リストを作成
image_list = make_list(DATASET_CSV, item='File Path', data_dir=DATA_DIR)

# 画像リストからランダムに N_SAMPLES 枚を取得
IDs = np.random.permutation(len(image_list))[:N_SAMPLES]
images = read_images(image_list, IDs)

##### PCAの実行
- 数分はかかると思います

In [ ]:
import numpy as np
from PCA_based.func import compress


# 平均を求める
mean = np.mean(images, axis=0, keepdims=True)

# 各データから平均を引く
X = images - mean # 平均を引いた後の images が行列 X に相当

# Dual PCAを実行
print('executing PCA ... ')
Cov = (1 / N_SAMPLES) * X @ X.T
eigenvalues, eigenvectors = np.linalg.eigh(Cov)
eigenvalues = eigenvalues[::-1] # 昇順に並んでいる固有値を降順に並び替え
eigenvectors = np.flip(eigenvectors, axis=1) # 固有ベクトルも同様に並び替え
eigenvalues = eigenvalues[:N_DIMS] # 上位 N_DIMS 個の固有値のみを取り出す
eigenvectors = eigenvectors[:, :N_DIMS] # 固有ベクトルについても同様
eigenvectors = eigenvectors.T @ X # Dual PCAによる固有ベクトルから本来のPCAによる固有ベクトルを復元
eigenvectors /= np.linalg.norm(eigenvectors, axis=1, keepdims=True) # 固有ベクトルのノルムを1にする
print('done')
print('')

# 各データを圧縮
compressed_data = compress(images, mean, eigenvectors)

# PCA結果をファイルに保存
np.save(RESULT_FILE_MEAN, mean)
np.save(RESULT_FILE_EIGENVECTORS, eigenvectors)
np.save(RESULT_FILE_COMPRESSED_DATA, compressed_data)
print('save mean vector as', RESULT_FILE_MEAN)
print('save eigenvectors as', RESULT_FILE_EIGENVECTORS)
print('save compressed data as', RESULT_FILE_COMPRESSED_DATA)
print('')

##### PCA実行結果ファイルのロード

In [5]:
import numpy as np


mean = np.load(RESULT_FILE_MEAN)
eigenvectors = np.load(RESULT_FILE_EIGENVECTORS)
compressed_data = np.load(RESULT_FILE_COMPRESSED_DATA)

##### 圧縮・復元のテスト

In [ ]:
import numpy as np
from PCA_based.func import read_single_image, show_single_image, compress, generate


# 画像を一枚ランダムに選択して読み込む
ID = np.random.randint(0, len(image_list))
img = read_single_image(image_list, ID)

# 読み込んだ画像を圧縮
vec = compress(img, mean, eigenvectors)

# 圧縮表現から画像を復元
gen = generate(vec, mean, eigenvectors)

# 圧縮前の画像を表示
show_single_image(img, shape=(H, W, C), window_name='original')

# 復元後の画像を表示
show_single_image(gen, shape=(H, W, C), window_name='reconstructed')

##### 乱数により定めた圧縮表現（ベクトル）からの画像生成

In [ ]:
import numpy as np
from PCA_based.func import show_images, generate


# 生成する画像の枚数
n_gen = 10

# 一様乱数を用いて適当なベクトルを作成（-20〜20 の範囲で）
random_range = [-20, 20] # 一様乱数の範囲
vec = (random_range[1] - random_range[0]) * np.random.rand(eigenvectors.shape[0], n_gen) + random_range[0]

# ベクトルから画像を生成
gen = generate(vec, mean, eigenvectors)

# 生成した画像を表示
show_images(gen, shape=(H, W, C), num=n_gen, num_per_row=5, window_name='generated')

##### GMM当てはめ処理の実行

In [ ]:
import pickle
from sklearn.mixture import GaussianMixture


# GMM当てはめを実行
print('fitting a GMM to the compressed data ... ')
gmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='spherical')
gmm.fit(compressed_data.T)
print('done')
print('')

# 結果をファイルに保存
pickle.dump(gmm, open(RESULT_FILE_GMM, 'wb'))

##### GMM当てはめ結果ファイルのロード

In [17]:
import pickle
from sklearn.mixture import GaussianMixture


gmm = pickle.load(open(RESULT_FILE_GMM, 'rb'))

##### GMMよりサンプリングした圧縮表現（ベクトル）からの画像生成

In [ ]:
from PCA_based.func import show_images, generate


# 生成する画像の枚数
n_gen = 10

# GMMからベクトルをサンプリング
vec = np.asarray(gmm.sample(n_samples=n_gen)[0]).T

# ベクトルから画像を生成
gen = generate(vec, mean, eigenvectors)

# 生成した画像を表示
show_images(gen, shape=(H, W, C), num=n_gen, num_per_row=5, window_name='generated')